# Eigenmodes, steady-state problems and efficient time integration

#### Job Marcelis, Ernani Hazbolatow, Koen Verlaan

In [ ]:
import numpy as np
from src.eigenmodes import *
from IPython.display import HTML

plt.rcParams['animation.embed_limit'] = 100

## 1: Eigenmodes of drums or membranes of different shapes

In this section, we investigate the wave equation of a 2D membrane with fixed boundary conditions along the edge. The 2D wave equation is given by:
\begin{equation}
    \frac{\partial^2 u}{\partial t^2} = c^2\nabla^2 u,
\end{equation}
where $u$ is the amplitude and c is the wave speed.

We look for a solution of the form:
\begin{equation}
    u(x,y,t) = v(x,y)T(t),
\end{equation}
where $v(x,y)$ is the spatial solution and $T(t)$ is the temporal solution.

To discretize $\nabla^2v(x, y) = Kv(x, y)$, we use a 5-point stencil, namely: $\frac{1}{h^2}(v_{i+1, j} + v_{i-1, j} + v_{i, j+1} + v_{i, j-1} -4v_{i,j})$. As an example, we use the following 3 by 3 system:

In [ ]:
draw_system_and_matrix(N=3, h=1, plot_system=True)

Since this is a 3 by 3 system, we have 9 gridpoints, meaning that our resulting matrix is 9 x 9. The entries in the matrix must placed in such a way that it corresponds to the 5-point stencil mentioned earlier:

In [ ]:
draw_system_and_matrix(N=3, h=1, print_latex_matrix=True, plot_system=False)

This matrix looks like:

$$
    \left[\begin{matrix}-4 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\1 & -4 & 1 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 1 & -4 & 0 & 0 & 1 & 0 & 0 & 0\\1 & 0 & 0 & -4 & 1 & 0 & 1 & 0 & 0\\0 & 1 & 0 & 1 & -4 & 1 & 0 & 1 & 0\\0 & 0 & 1 & 0 & 1 & -4 & 0 & 0 & 1\\0 & 0 & 0 & 1 & 0 & 0 & -4 & 1 & 0\\0 & 0 & 0 & 0 & 1 & 0 & 1 & -4 & 1\\0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & -4\end{matrix}\right]
$$

We have implemented 3 different shapes for the drums/membranes. For each shape the matrix is constructed slightly differently and the equation $Mv = Kv$ is solved using `scipy.sprase.linalg.eigs`. This was done because the matrices are sparse by nature and the calculation would therefore be slow when using `scipy` functions designed for dense matrices. Below we plot the eigenmodes for some of the smallest eigenfrequencies for each shape.

Square:

In [ ]:
L = 1
h = 0.01
N = int(L/h) - 1
M_square = construct_M_square(N, h)
lamda, v = linalg.eigs(M_square, k=4, which='SM')
plot_eigenvectors(lamda, v, grid_shape=(N,N), L=L, plot_title='Eigenmodes of Square Membrane')

Rectangle:

In [ ]:
L = 1
hx = 0.01
hy = 0.02
Nx = int(L/hx) - 1
Ny = int(2*L/hy) - 1
M_rec = construct_M_rec(Nx, Ny, hx, hy)
lamda, v = linalg.eigs(M_rec, k=4, which='SM')
plot_eigenvectors(lamda, v, grid_shape=(Ny, Nx), L=L, shape='rectangle', plot_title='Eigenmodes of Rectangular Membrane')

Circle:

In [ ]:
M_circle = construct_M_circle(N, h, L)
lamda, v = linalg.eigs(M_circle, k=4, which='SM')
plot_eigenvectors(lamda, v, grid_shape=(N,N), L=L, plot_title='Eigenmodes of Circular Membrane')

As mentioned above, we use `eigs()` to compute the eigenvalues and eigenvector since the matrix is sparse. To obtain insight in how much faster `eigs()` is compared to `eig()`, we test the performance on a square grid with different system sizes. Each system size is done 25 times to obtain an average with confidence intervals.

In [ ]:
Ns = np.linspace(10, 30, 15)
num_runs = 25
mean_eig, CI_eig, mean_eigs, CI_eigs = performance_compare(Ns, num_runs, plot=True)

We now investigate what happends when the system size L is varied by plotting the eigenfrequencies for each shape as a function of L.

In [ ]:
Ls = np.linspace(0.5, 2, 10)
h = 0.01

eig_freq_sq = spectrum_vs_L(Ls, h, 'square')
eig_freq_rec = spectrum_vs_L(Ls, h, 'rectangle')
eig_freq_cir = spectrum_vs_L(Ls, h, 'circle')

plot_spectrum_vs_L(Ls, eig_freq_sq, eig_freq_rec, eig_freq_cir)

We also investigate what happens when the number of discretization steps is changed.

In [ ]:
Ns = np.linspace(3, 50, 10)
L = 1.0

freq_vs_N_sq = spectrum_vs_num_steps(Ns, L, 'square')
freq_vs_N_rec = spectrum_vs_num_steps(Ns, L, 'rectangle')
freq_vs_N_cir = spectrum_vs_num_steps(Ns, L, 'circle')

plot_spectrum_vs_num_steps(Ns, freq_vs_N_sq, freq_vs_N_rec, freq_vs_N_cir)

In addition to the solution of the eigenvalue problem, we add the time component. We then have a solution of the form: $u(x,y,t) = v(x,y)T(t)$, where $T(t) = Acos(c\lambda t) + Bsin(c\lambda t)$. To simplify, we choose $c = 1$, $A = 1$, and $B=0$.

In [ ]:
c = 1
L = 1
h = 0.01
N = int(L/h) - 1
total_time = np.linspace(0, 20*np.pi, 100)
M_square = construct_M_square(N, h)
lamda, v = linalg.eigs(M_square, k=8, which='SM')
idx = np.flip(np.argsort(np.real(lamda)))
lamda = lamda[idx]
v = v[:, idx]

In [ ]:
ani1 = time_dependent_modes(total_time, c, lamda, v, N, mode_number=4)
HTML(ani1.to_jshtml())

In [ ]:
ani2 = time_dependent_modes(total_time, c, lamda, v, N, mode_number=6)
HTML(ani2.to_jshtml())